### This is a simple notebook to train a Decision Tree to discriminate between two types of collisional events.

Data for this exercise were kindly provided by [Sascha Caron](https://www.nikhef.nl/~scaron/).

Author: Viviana Acquaviva

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
rc('text', usetex=False)

### The first part of this notebook is to show the manipulation I did to get the data in the format we need and to select a random sample to keep things more manageable.

You don't need to run it!

In [ ]:
df = pd.read_csv('TrainingValidationData.csv', delimiter=',', names=list(['P'+str(i) for i in range(53)]) )
print(df.columns)

In [ ]:
df.head()

In [ ]:
new = df['P0'].str.split(';',expand=True)

new.columns = ['numID', 'processID', 'weight', 'MET', 'METphi', 'Type_1']

In [ ]:
new.head()

In [ ]:
df = df.join(new, how='outer') #join them side to side

In [ ]:
df.head()

#### The new columns have been appended at the end; we still need to split the type of product.

In [ ]:
for i in range(4,53,4):

    new = df['P'+str(i)].str.split(';',expand=True) 
    
    df['P'+str(i)] = new[0]
    
    df['Type_'+str(int(i/4+1))] = new[1]
    
print(df.columns)

In [ ]:
df = df.drop('P0', axis=1)

In [ ]:
df.columns.values

In [ ]:
#just re-ordering

cols = ['numID', 'processID', 'weight',
       'MET', 'METphi', 'Type_1', 'P1', 'P2', 'P3', 'P4',  'Type_2', 'P5', 'P6', 'P7', 'P8', 'Type_3', 'P9', 'P10', 'P11',
       'P12',  'Type_4', 'P13', 'P14', 'P15', 'P16', 'Type_5','P17', 'P18', 'P19', 'P20',
       'Type_6','P21', 'P22', 'P23', 'P24', 'Type_7','P25', 'P26', 'P27', 'P28', 'Type_8','P29',
       'P30', 'P31', 'P32', 'Type_9', 'P33', 'P34', 'P35', 'P36', 'Type_10','P37', 'P38',
       'P39', 'P40', 'Type_11', 'P41', 'P42', 'P43', 'P44', 'Type_12', 'P45', 'P46', 'P47',
       'P48', 'Type_13','P49', 'P50', 'P51', 'P52']

In [ ]:
X = df[cols].drop(['numID', 'processID', 'weight'], axis = 1)

In [ ]:
len(cols)

In [ ]:
X.head() 

In [ ]:
X.describe() #remember that this only affects numerical columns

Some columns that should be numerical are of type "object"

In [ ]:
X.columns[X.dtypes == object]

Re-cast data type where appropriate

In [ ]:
for el in ['MET', 'METphi', 'P4', 'P8', 'P12',
        'P16',  'P20', 'P24',  'P28',
    'P32', 'P36', 'P40', 'P44',
      'P48', 'P52']:
    X[el] = X[el].astype('float64')

In [ ]:
X.dtypes

#### Select 5000 rows

In [ ]:
np.random.seed(10)

sel = np.random.choice(df.shape[0], 5000)

features = X.iloc[sel,:]

In [ ]:
features.shape

In [ ]:
features.columns

Reset index

In [ ]:
features.reset_index(drop=True, inplace=True)

In [ ]:
features.head()

#### Export feature data frame to file

In [ ]:
features.to_csv('ParticleID_features.csv', index_label= 'ID')

#### Select labels

In [ ]:
y = df.processID[sel].values # values makes it an array

In [ ]:
y

#### Export labels to file

In [ ]:
np.savetxt('ParticleID_labels.txt', y, fmt = '%s')

## Start here!

Read in features and labels.

In [ ]:
features = pd.read_csv('ParticleID_features.csv', index_col='ID')

In [ ]:
features.head(10)

In [ ]:
features.shape

In [ ]:
y = np.genfromtxt('ParticleID_labels.txt', dtype = str)

In [ ]:
y

#### We need to turn categorical (string-type) labels into an array, e.g. 0/1.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #turns categorical into 1 ... N

In [ ]:
y

In [ ]:
y = le.fit_transform(y)

In [ ]:
y #This uses 1 for the first instance, I actually wanted  4top to be my positive label.

In [ ]:
target = np.abs(y - 1)

In [ ]:
target # Happier now.

#### Let's take a look at these features, using the "describe" property.

In [ ]:
features.describe() #Note that this automatically excludes non-numerical type columns

### Important:

Looking at the "count" row, we can see that the whole data set has 5,000 rows, but some columns are present only for a fraction of them. This is because of the variable number of products in each collision.

#### Option 1: Only consider first 16 columns (first four products) so we have limited imputing/manipulation problems.

In [ ]:
features_lim = features[['MET', 'METphi', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12',  'P13', 'P14', 'P15', 'P16']]

In [ ]:
features_lim.head(20)

In [ ]:
features_lim.describe() #This automatically excludes non-numerical type columns, and missing values/NaNs are not counted.

There are still some feature columns with different length! This means there might be NaN values. Let's replace them with 0 for the moment (note: this is the worst possible imputing strategy, but since we are applying it to a small percentage of the data set, we can get away with it). 

Fill with 0 everywhere there is a NaN:

In [ ]:
features_lim = features_lim.fillna(0) #Fill with 0 everywhere there is a NaN

#### Let's see what "describe" says now.

In [ ]:
features_lim.describe()

Yay - we now have consistent sizes, so we can use these as feature arrays, BUT be mindful of possible negative impacts of our imputing strategies.

### Let's move onto a quick exploration of labels.

In [ ]:
np.sum(target)/len(target) #distribution (helps with benchmarking!)

84\% in the negative label, 16\% in the positive label. A bit unbalanced; a classifier that puts everything in the negative class will have 84\% accuracy.

#### How about a random classifier that just assigns a random class according to class distribution?

In [ ]:
#Numerical solution

acc=0
for i in range(1000):
    x = np.random.choice(target,5000)
    acc += metrics.accuracy_score(target,x)
print(acc/1000)

#Analytic solution

print(0.8378*(0.8378) + 0.1622*0.1622)

### Let's start with a decision tree.

Establish benchmark.

In [ ]:
bmodel = DecisionTreeClassifier(random_state=10) 

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=101) 

In [ ]:
l_benchmark_lim = cross_validate(bmodel, features_lim, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
l_benchmark_lim

In [ ]:
np.round(l_benchmark_lim['test_score'].mean(),3), np.round(l_benchmark_lim['test_score'].std(), 3)

We can also check the predicted labels. Cross\_val\_predict will compile labels predicted when each object was in the test fold.

In [ ]:
ypred_bench_lim = cross_val_predict(bmodel, features_lim, target, cv = cv)

Slightly better than a super silly classifier that says no to everything.

In fact, we can look at the confusion matrix to see what is happening:

In [ ]:
metrics.confusion_matrix(target, ypred_bench_lim)

### Learning curves 

Note: Import function at the end of the notebook.

In [ ]:
plot_learning_curve(bmodel, 'Learning Curves, DT model', features_lim, target, train_sizes = np.array([0.05,0.1,0.2,0.5,1.0]), cv = KFold(n_splits=5, shuffle=True));

### Conclusions

Our classifier is behaving .... than a random/lazy one.

Our model suffers from ....

Having more data would .....

### Parameter optimization 

(note: this is NOT nested cross validation).

In [ ]:
model = DecisionTreeClassifier(random_state=10) 

model.get_params() #this shows how we can access parameters 

#### Tree Parameters

Examples of hyperparameters to play with are:

-  The minimum number of instances in a leaf node;

-  The minimum number of instances required in a split node;

- The maximum depth of tree.

-  The criterion chosen to decide whether a split is "worth it", expressed in terms of information gain.

### We can define a grid of parameter values to run the optimization. 

(should do nested CV to estimate generalization error!)

Note that this might take a while.

In [ ]:
model = DecisionTreeClassifier(random_state=10, class_weight={1:10})
scores = cross_validate(model, features_lim, target, return_train_score=True, cv = StratifiedKFold(n_splits=5, shuffle=True))
print(scores)

In [ ]:
features_lim.shape

In [ ]:
#optimizing ST: THIS IS NOT YET NESTED CV

bmodel = DecisionTreeClassifier(random_state=10)

parameters = {'max_depth':[6, 10, None], \
              'class_weight': [{1:1}, {1:5}, {1:10}], \
              'max_features':[5, 8, 12], \
              'min_samples_leaf': [1, 4, 8]}

model = GridSearchCV(bmodel, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)

model.fit(features_lim,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

#### Visualize the scores in a data frame, and rank them according to test scores.

I like to look at mean, std of test scores, mean of train scores (so I can evaluate if they differ and the significance of the result), and also fitting time (would pick a model that takes less time if scores are comparable).

In [ ]:
scores_lim = pd.DataFrame(model.cv_results_)

scores_lim[['params','mean_test_score','std_test_score','mean_train_score', \
            'mean_fit_time']].sort_values(by = 'mean_test_score', ascending = False)

#### Questions I like to answer here:

Which parameters matter (are constant across the top models) and which don't?

Is the best model at the edge of the parameter grid I am considering (and if so, should I explore a larger parameter space)?


### Final diagnosis 

We have reduced the variance and obtained higher test scores (hooray!)

Can we now also explore the bias issue?

We can try two things: making up new features which might help, based on what we know about the problem, and using an imputing strategy to include information about the discarded features. We will only explore the first one.

### Next step: define some new variables. 

In [ ]:
features = features.fillna(0) #takes care of nan

In [ ]:
features = features.replace('', 0) #takes care of empty string values

In [ ]:
features.head()

In [ ]:
np.unique(features.Type_1.values)

Let's start by looking at what kind of particles we have as a product of the collision.

In [ ]:
np.unique(np.array([features['Type_'+str(i)].values for i in range(1,14)]).astype('str'))

Here are the proposed new features (justification can be found in the book!)
    
    1. The total number of particles produced
    2. The total number of b jets
    3. The total number of jets
    4. The total number of leptons (electrons, positron, mu+, mu-)

In [ ]:
#count number of non-zero types 

ntot = np.array([-(np.sum(np.array([features['Type_'+str(i)].values[j] == 0 for i in range(1,14)])) - 13) for j in range(features.shape[0])])

In [ ]:
#define new column in my data frame

features['Total_products'] = ntot

In [ ]:
#count number of b jets 

nbtot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'b' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
#define new column in my data frame

features['Total_b'] = nbtot

In [ ]:
#Actually, let's count all types (jets, photons g, e-, e+, mu-, mu+)

njtot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'j' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
ngtot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'g' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_el_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'e-' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_pos_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'e+' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_muneg_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'm-' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_mupos_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'm+' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_lepton_tot = n_el_tot + n_pos_tot + n_muneg_tot + n_mupos_tot

And here we define the other new features:

In [ ]:
features['Total_j'] = njtot
features['Total_g'] = ngtot
features['Total_leptons'] = n_lepton_tot

In [ ]:
features.head()

### Feature engineering 1: impact of ad-hoc variables

In [ ]:
features_lim_2 = features[['MET', 'METphi', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12',  'P13', 'P14', 'P15', 'P16','Total_products', 'Total_b' ,'Total_j','Total_g', 
              'Total_leptons']]

In [ ]:
bmodel #remember our benchmark model?

In [ ]:
benchmark_lim2 = cross_validate(bmodel, features_lim_2, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
benchmark_lim2

In [ ]:
np.round(benchmark_lim2['test_score'].mean(),3), np.round(benchmark_lim2['test_score'].std(), 3)

We can optimize this model as well; it will take a while, just like the previous time.

In [ ]:
#optimizing DT

bmodel = DecisionTreeClassifier(random_state=10)

parameters = {'max_depth':[4, 6, 10, None], \
              'class_weight': [{1:1}, {1:5}, {1:10}], \
              'max_features':[5, 8, 12], \
              'min_samples_leaf': [1, 4, 8]}

model = GridSearchCV(bmodel, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)

model.fit(features_lim_2,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores_lim_2 = pd.DataFrame(model.cv_results_)
scores_lim_2[['params','mean_test_score','std_test_score','mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)

### Another feature engineering attempt we could potentially do is use the type of product in the i-th location as a feature.

We could do it with label encoding, but this introduces a notion of distance metric (labels that are mapped to 0 and 1 are interpreted to be closer to each other than labels that are mapped into 0 and 7).

We introduce as many new columns as categorical labels, and we just use a 0/1 to indicate that the particle is of that type.

In [ ]:
features_add = pd.get_dummies(data=features, columns=['Type_'+str(i) for i in range(1,14)])

In [ ]:
features_add.columns[58:80]

In [ ]:
features_add.shape

### Feature engineering 2: add other variables (type of product)

In [ ]:
features_lim_3 = features_add[['MET', 'METphi', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12',  'P13', 'P14', 'P15', 'P16','Total_products', 'Total_b' ,'Total_j','Total_g', 
              'Total_leptons','Type_1_b',
       'Type_1_j', 'Type_2_0', 'Type_2_b', 'Type_2_e+', 'Type_2_e-',
       'Type_2_g', 'Type_2_j', 'Type_2_m+', 'Type_2_m-', 'Type_3_0',
       'Type_3_b', 'Type_3_e+', 'Type_3_e-', 'Type_3_g', 'Type_3_j',
       'Type_3_m+', 'Type_3_m-', 'Type_4_0', 'Type_4_b', 'Type_4_e+',
       'Type_4_e-', 'Type_4_g', 'Type_4_j', 'Type_4_m+', 'Type_4_m-']]

In [ ]:
features_lim_3.head()

In [ ]:
benchmark = cross_validate(bmodel, features_lim_3, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
benchmark

In [ ]:
np.round(benchmark['test_score'].mean(),3), np.round(benchmark['test_score'].std(), 3)

In [ ]:
np.round(benchmark['train_score'].mean(),3), np.round(benchmark['train_score'].std(), 3)

#### No further improvement is observed, although we should optimize the model.

In [ ]:
#optimizing DT 

bmodel = DecisionTreeClassifier(random_state=10)

parameters = {'max_depth':[4, 6, 10, None], \
              'class_weight': [{1:1}, {1:5}, {1:10}], \
              'max_features':[5, 8, 12], \
              'min_samples_leaf': [1, 4, 8]}

model = GridSearchCV(bmodel, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)

model.fit(features_lim_3,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

scores_lim_3 = pd.DataFrame(model.cv_results_)
scores_lim_3[['params','mean_test_score','mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)

### Some conclusions

1. Feature engineering often works best if we use subject matter knowledge;

2. Better features > more features;

3. The procedure above is correct for finding the optimal hyperparameters, but we might be a bit over-optimistic if we take the test scores for the optimal model as representative of the generalization error. Instead, we should have a 3-tier structure (train/validation/test), where we use the validation set to choose parameters, and the test set to evaluate the generalization error. If we do it in cross validation, this is called nested cross validation (see book).

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=5,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring = 'accuracy', scale = False):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("# of training examples",fontsize = 14)
 
    plt.ylabel("Accuracy score",fontsize = 14)
    
    if (scale == True):
        scaler = sklearn.preprocessing.StandardScaler()
        X = scaler.fit_transform(X)
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
#    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="b")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="b",
             label="Training score from CV")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Test score from CV")

    plt.legend(loc="best",fontsize = 12)
    return plt